# Subprepare

Gates for preparing coefficient states.

In section III.D. of the [Linear T paper](https://arxiv.org/abs/1805.03662) the authors introduce
a technique for initializing a state with $L$ unique coefficients (provided by a classical
database) with a number of T gates scaling as 4L + O(log(1/eps)) where eps is the
largest absolute error that one can tolerate in the prepared amplitudes.

In [ ]:
import cirq
import numpy as np
import cirq_qubitization
import cirq_qubitization.testing as cq_testing
from cirq_qubitization.jupyter_tools import display_gate_and_compilation
from typing import *

## `GenericSubPrepare`
Initialize a state with $L$ unique coefficients.

In particular, we take the zero state to:

$$
\sum_{\ell=0}^{L-1} \sqrt{p_\ell} |\ell\rangle |\mathrm{temp}_\ell\rangle
$$

where the probabilities $p_\ell$ are $\mu$-bit binary approximations to the true values and
where the temporary register must be treated with care, see the details in Section III.D. of
the reference.

The preparation is equivalent to [classical alias sampling](https://en.wikipedia.org/wiki/Alias_method):
we sample `l` with probability `p[l]` by first selecting `l` uniformly at random and then
returning it with probability `keep[l] / 2**mu`; otherwise returning `alt[l]`.

Registers:
    selection: The input/output register $|\ell\rangle$ of size lg(L) where the desired
        coefficient state is prepared.
    temp: Work space comprised of sub registers:
        - sigma: A mu-sized register containing uniform probabilities for comparison against
            `keep`.
        - alt: A lg(L)-sized register of alternate indices
        - keep: a mu-sized register of probabilities of keeping the initially sampled index.
        - one bit for the result of the comparison.
    selection_ancilla: Ancilla space for QROM loading.

This gate corresponds to the following operations:
 - UNIFORM_L on the selection register
 - H^mu on the sigma register
 - QROM addressed by the selection register into the alt and keep registers.
 - LessThanEqualGate comparing the keep and sigma registers.
 - Coherent swap between the selection register and alt register if the comparison
   returns True.

Total space will be (2 * log(L) + 2 mu + 1) work qubits + log(L) ancillas for QROM.
The 1 ancilla in work qubits is for the `LessThanEqualGate` followed by coherent swap.

#### Parameters
 - `lcu_probabilities`: The LCU coefficients.
 - `probability_epsilon`: The desired accuracy to represent each probability (which sets mu size and keep/alt integers). See `openfermion.circuits.lcu_util.preprocess_lcu_coefficients_for_reversible_sampling` for more information. 

#### References
[Encoding Electronic Spectra in Quantum Circuits with Linear T Complexity](https://arxiv.org/abs/1805.03662). Babbush et. al. (2018). Section III.D. and Figure 11.


In [ ]:
from cirq_qubitization.generic_subprepare import GenericSubPrepare

coeffs = np.array([1.0, 1, 3, 2])
mu = 3

g = cq_testing.GateHelper(
    GenericSubPrepare(coeffs, probability_epsilon=2**-mu / len(coeffs))
)

display_gate_and_compilation(g)

In [ ]:
# The "alt" indices are log(L) bits like the "selection" and "alt" registers.
g.gate.alternates_bitsize

In [ ]:
# The "keep" probabilities are mu bits like the "keep" and "sigma" registers.
g.gate.keep_bitsize